In [20]:
import torch
import sys
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from deeprobust.graph.global_attack import MetaApprox, Metattack
from deeprobust.graph.utils import *
from deeprobust.graph.defense import *
from deeprobust.graph.data import Dataset
import argparse
from scipy.sparse import csr_matrix
import pickle
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import normalize
import scipy
import numpy as np
from defense import GCN_attack

# parser = argparse.ArgumentParser()
# parser.add_argument('--no-cuda', action='store_true', default=False,
#                     help='Disables CUDA training.')
# parser.add_argument('--seed', type=int, default=15, help='Random seed.')
# parser.add_argument('--epochs', type=int, default=200,
#                     help='Number of epochs to train.')
# parser.add_argument('--lr', type=float, default=0.01,
#                     help='Initial learning rate.')
# parser.add_argument('--weight_decay', type=float, default=5e-4,
#                     help='Weight decay (L2 loss on parameters).')
# parser.add_argument('--hidden', type=int, default=16,
#                     help='Number of hidden units.')
# parser.add_argument('--dropout', type=float, default=0.5,
#                     help='Dropout rate (1 - keep probability).')
# parser.add_argument('--dataset', type=str, default='cora', choices=['cora', 'cora_ml', 'citeseer', 'polblogs', 'pubmed'], help='dataset')
# parser.add_argument('--ptb_rate', type=float, default=0.05,  help='pertubation rate')
# parser.add_argument('--model', type=str, default='Meta-Self', choices=['A-Meta-Self', 'Meta-Self'], help='model variant')

# parser.add_argument('--modelname', type=str, default='GCN',  choices=['GCN', 'GAT','GIN', 'JK'])
# parser.add_argument('--defensemodel', type=str, default='GCNJaccard',  choices=['GCNJaccard', 'RGCN', 'GCNSVD'])
# parser.add_argument('--GNNGuard', type=bool, default=False,  choices=[True, False])

# args = parser.parse_args()

SEED = 15
DATASET = 'cora'
PTB_RATE = 0.05
MODEL = 'Meta-Self'
GNNGUARD = False
MODELNAME = 'GCN'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(SEED)
torch.manual_seed(SEED)
if device != 'cpu':
    torch.cuda.manual_seed(SEED)

data = Dataset(root='./Datasets/', name=DATASET)

adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

# 判断稀疏矩阵
if scipy.sparse.issparse(features)==False:
    features = scipy.sparse.csr_matrix(features)

perturbations = int(PTB_RATE * (adj.sum()//2))
adj, features, labels = preprocess(adj, features, labels, preprocess_adj=False)

#1. to CSR sparse
adj, features = csr_matrix(adj), csr_matrix(features)

"""add undirected edges, orgn-arxiv is directed graph, we transfer it to undirected closely following 
https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-arxiv
"""
adj = adj + adj.T
adj[adj>1] = 1

modified_adj = pickle.load(open('saved_pk/mettack_modified_adj_cora2484_0.05fake.pk', 'rb'))
modified_adj = csr_matrix(modified_adj)
attention = GNNGUARD # if True, our method; if False, run baselines


Loading cora dataset...
Selecting 1 largest connected components


In [2]:
# Setup GCN as the Surrogate Model
surrogate = GCN_attack(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=16,
        dropout=0.5, with_relu=False, with_bias=True, weight_decay=5e-4, device=device)

surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train, train_iters=201)

# Setup Attack Model
if 'Self' in MODEL:
    lambda_ = 0
if 'Train' in MODEL:
    lambda_ = 1
if 'Both' in MODEL:
    lambda_ = 0.5

if 'A' in MODEL:
    model = MetaApprox(model=surrogate, nnodes=adj.shape[0], feature_shape=features.shape, attack_structure=True, attack_features=False, device=device, lambda_=lambda_)

else:
    model = Metattack(model=surrogate, nnodes=adj.shape[0], feature_shape=features.shape,  attack_structure=True, attack_features=False, device=device, lambda_=lambda_)

model = model.to(device)

attention = GNNGUARD # if True, our method; if False, run baselines

# save the mettacked adj

model.attack(features, adj, labels, idx_train, idx_unlabeled, perturbations, ll_constraint=False)
modified_adj = model.modified_adj
pickle.dump(modified_adj, open('saved_pk/mettack_modified_adj_cora2484_0.05fake.pk', 'wb'))
print('modified_adj saved')


Perturbing graph:   0%|          | 0/253 [00:00<?, ?it/s]

GCN loss on unlabled data: 0.5256392359733582
GCN acc on unlabled data: 0.8270004470272687
attack loss: 0.34636449813842773


C:\Users\Banana\.conda\envs\torch-py37\lib\site-packages\deeprobust\graph\utils.py:541: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  rows = index // array_shape[1]
Perturbing graph:   0%|          | 1/253 [00:08<34:56,  8.32s/it]

GCN loss on unlabled data: 0.5196269750595093
GCN acc on unlabled data: 0.8363880196691998
attack loss: 0.3322211503982544


Perturbing graph:   1%|          | 2/253 [00:16<35:03,  8.38s/it]

...

GCN loss on unlabled data: 1.4231394529342651
GCN acc on unlabled data: 0.7179257934734019
attack loss: 1.4961013793945312


Perturbing graph: 100%|█████████▉| 252/253 [34:17<00:08,  8.02s/it]

GCN loss on unlabled data: 1.3907556533813477
GCN acc on unlabled data: 0.7165847116674117
attack loss: 1.4691990613937378


Perturbing graph: 100%|██████████| 253/253 [34:25<00:00,  8.16s/it]


FileNotFoundError: [Errno 2] No such file or directory: 'saved_pk/mettack_modified_adj_cora2484_0.05fake.pk'

In [21]:

def test(adj):
    # """defense models"""
    # classifier = globals()[args.defensemodel](nnodes=adj.shape[0], nfeat=features.shape[1], nhid=16,
    #                                           nclass=labels.max().item() + 1, dropout=0.5, device=device)

    ''' testing model '''
    classifier = globals()[MODELNAME](nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1, dropout=0.5, device=device)

    classifier = classifier.to(device)

    classifier.fit(features, adj, labels, idx_train, train_iters=201,
                   idx_val=idx_val,
                   idx_test=idx_test,
                   verbose=True, 
                   attention=attention)
                    # idx_val=idx_val, idx_test=idx_test , model_name=model_name
    classifier.eval()

    # classifier.fit(features, adj, labels, idx_train, idx_val) # train with validation model picking
    acc_test = classifier.test(idx_test)
    print(acc_test)
    # return acc_test.item()

test(adj)

test(modified_adj)

=== training gcn model ===
Epoch 0, training loss: 1.9819684028625488
Epoch 10, training loss: 1.1804733276367188
Epoch 20, training loss: 0.5908041596412659
Epoch 30, training loss: 0.3222294747829437
Epoch 40, training loss: 0.15581312775611877
Epoch 50, training loss: 0.11106248199939728
Epoch 60, training loss: 0.08716487139463425
Epoch 70, training loss: 0.08035001158714294
Epoch 80, training loss: 0.07265067845582962
Epoch 90, training loss: 0.09728260338306427
Epoch 100, training loss: 0.08143855631351471
Epoch 110, training loss: 0.0835583433508873
Epoch 120, training loss: 0.06565006077289581
Epoch 130, training loss: 0.042687300592660904
Epoch 140, training loss: 0.039915770292282104
Epoch 150, training loss: 0.06322012096643448
Epoch 160, training loss: 0.0405743308365345
Epoch 170, training loss: 0.043181657791137695
Epoch 180, training loss: 0.046466391533613205
Epoch 190, training loss: 0.03725040704011917
Epoch 200, training loss: 0.05468853563070297
=== picking the best